In [ ]:
import numpy as np
from PIL import Image
from svglib.svglib import svg2rlg
from reportlab.graphics import renderPM
import io

def rasterize_svg(svg_content, width, height):
    """Rasterize SVG content to a numpy array."""
    drawing = svg2rlg(io.StringIO(svg_content))
    img_data = renderPM.drawToString(drawing, fmt="PNG", dpi=72*(width/drawing.width))
    img = Image.open(io.BytesIO(img_data))
    img = img.resize((width, height), Image.LANCZOS)
    return np.array(img)

def evaluate_occlusion(solution_svg, ground_truth_svg, width=1024, height=1024):
    """Evaluate occlusion by comparing rasterized SVGs."""
    # Rasterize both SVGs
    solution_raster = rasterize_svg(solution_svg, width, height)
    ground_truth_raster = rasterize_svg(ground_truth_svg, width, height)
    
    # Convert to binary (assuming white background, any non-white pixel is part of the shape)
    solution_binary = (solution_raster[:,:,:3] != 255).any(axis=2)
    ground_truth_binary = (ground_truth_raster[:,:,:3] != 255).any(axis=2)
    
    # Calculate pixel-wise agreement
    agreement = np.sum(solution_binary == ground_truth_binary)
    total_pixels = width * height
    
    # Calculate accuracy
    accuracy = agreement / total_pixels
    
    # Calculate IoU (Intersection over Union)
    intersection = np.sum(np.logical_and(solution_binary, ground_truth_binary))
    union = np.sum(np.logical_or(solution_binary, ground_truth_binary))
    iou = intersection / union if union > 0 else 0
    
    return {
        'accuracy': accuracy,
        'iou': iou
    }

# Example usage
solution_svg = """
<svg width="100" height="100">
    <rect x="10" y="10" width="80" height="80" fill="black" />
</svg>
"""

ground_truth_svg = """
<svg width="100" height="100">
    <rect x="20" y="20" width="60" height="60" fill="black" />
</svg>
"""

result = evaluate_occlusion(solution_svg, ground_truth_svg)
print(f"Accuracy: {result['accuracy']:.4f}")
print(f"IoU: {result['iou']:.4f}")